# 1. ResNet-50 - Transfer learning + own output layer
In this model we use the ResNet-50 structure with including the weights of 'imagenet'. To customize this to my own problem I removed the dense layer and added a dense layer of 7 nodes.

## 1.1 Import libaries, general functions and global variables

### 1.1.1 Libaries

In [17]:
# Import all the libaris in the module dependencies
from dependencies import *

### 1.1.2 General functions

In [18]:
# Import some general functions created by myself (loading the data, visualizing conv layers, etc.)
from general_functions import *

### 1.1.3 Global variables

In [19]:
dataset_path = os.path.join(os.getcwd(), 'datasets', 'UTKface_cropped-new-structure-balanced')

In [20]:
training_path   = os.path.join(os.getcwd(), 'datasets', 'UTKface_cropped-new-structure-balanced', 'training')
validation_path = os.path.join(os.getcwd(), 'datasets', 'UTKface_cropped-new-structure-balanced', 'validation')
test_path       = os.path.join(os.getcwd(), 'datasets', 'UTKface_cropped-new-structure-balanced', 'test')

In [21]:
df = load_dataset('df-UTKface_cropped-balanced.pkl')
df.head()

,age,age-binned,gender,race,datetime,category,path
0,10,"(0, 10]",0,0,20161220222308131,training,C:\Users\Paulo\Documents\GitHub\deep-learning-age-estimation\notebooks\datasets\UTKface_cropped-new-structure-balanced\training\10
1,10,"(0, 10]",0,0,20170103200329407,training,C:\Users\Paulo\Documents\GitHub\deep-learning-age-estimation\notebooks\datasets\UTKface_cropped-new-structure-balanced\training\10
2,10,"(0, 10]",0,0,20170103200522151,training,C:\Users\Paulo\Documents\GitHub\deep-learning-age-estimation\notebooks\datasets\UTKface_cropped-new-structure-balanced\training\10
3,10,"(0, 10]",0,0,20170103233459275,training,C:\Users\Paulo\Documents\GitHub\deep-learning-age-estimation\notebooks\datasets\UTKface_cropped-new-structure-balanced\training\10
4,10,"(0, 10]",0,0,20170104013211746,training,C:\Users\Paulo\Documents\GitHub\deep-learning-age-estimation\notebooks\datasets\UTKface_cropped-new-structure-balanced\training\10


In [22]:
df['age-binned'].value_counts()

(60, 70]    1170
(50, 60]    1170
(40, 50]    1170
(30, 40]    1170
(20, 30]    1170
(10, 20]    1170
(0, 10]     1170
Name: age-binned, dtype: int64

## 1.2 Generator

In [23]:
# Define image generators objects
train_datagen      = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen       = ImageDataGenerator(rescale=1./255)

# Create image generators
TARGET_SIZE = (224, 224) # We replicate the 224 x 224 shape like the paper.
COLOR_MODE  = 'rgb'      # Model ResNet-50 will expect rgb
BATCH_SIZE  = 32

train_generator    = train_datagen.flow_from_directory(directory   = training_path,
                                                       target_size = TARGET_SIZE,
                                                       color_mode  = COLOR_MODE,
                                                       batch_size  = BATCH_SIZE)

validation_datagen = train_datagen.flow_from_directory(directory   = validation_path,
                                                       target_size = TARGET_SIZE,
                                                       color_mode  = COLOR_MODE,
                                                       batch_size  = BATCH_SIZE)

test_generator     = test_datagen.flow_from_directory(directory    = test_path,
                                                      target_size  = TARGET_SIZE,
                                                      color_mode   = COLOR_MODE,
                                                      batch_size   = BATCH_SIZE)

Found 6177 images belonging to 7 classes.
Found 1629 images belonging to 7 classes.
Found 384 images belonging to 7 classes.


<br>

## 1.3 Import ResNet-50 (including weights) and add output layer with 7 nodes

In [40]:
TARGET_SIZE_WITH_DIMENSIONS = list(TARGET_SIZE)
TARGET_SIZE_WITH_DIMENSIONS.append(3)
TARGET_SIZE_WITH_DIMENSIONS = tuple(TARGET_SIZE_WITH_DIMENSIONS)

model = keras.applications.resnet.ResNet50(include_top = False, 
                                           weights     = 'imagenet',  
                                           input_shape = TARGET_SIZE_WITH_DIMENSIONS,
                                           classes     = 7)

In [41]:
# Freeze everything from the ResNet-50 model
for layer in model.layers:
    layer.trainable = False

In [42]:
# Adding GlobalAveragePooling2D and an output layer
x = model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(7, activation= 'softmax')(x)
model = Model(inputs = model.input, outputs = predictions)

# Add compiler
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy']) 

In [43]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

<br>

## 1.4 Run the model

In [36]:
filepath = "models-and-results-UTKface_cropped-balanced/resnet-50-1.0-{val_accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit_generator(generator           = train_generator,
                              steps_per_epoch     = train_generator.n // train_generator.batch_size,
                              epochs              = 15,
                              validation_data     = validation_datagen,
                              validation_steps    = validation_datagen.n // validation_datagen.batch_size,
                              verbose             = 1,
                              callbacks           = callbacks_list,
                              workers             = (multiprocessing.cpu_count())
                             )

Epoch 1/15
  2/193 [..............................] - ETA: 1:15:20 - loss: 1.9456 - accuracy: 0.1406

KeyboardInterrupt: 

## 1.5 Save the results

In [ ]:
# Save results
model.save('models-and-results-UTKface_cropped-balanced/neural-network-resnet-50-1.0-model.h5')
save_history_model(history          = history,
                   filename         = 'neural-network-resnet-50-1.0-history', 
                   storage_location = 'models-and-results-UTKface_cropped-balanced')

In [ ]:
# Plot history
plot_history(history, line1='accuracy', line2='val_accuracy',
             x_as = 'Epochs', y_as = 'Accuracy', title = 'neural-network-1.0-resnet-50 Accuracy')

plot_history(history, line1='loss', line2='val_loss',
             x_as = 'Epochs', y_as = 'Loss', title = 'neural-network-1.0-resnet-50 Loss')

In [ ]:
# Plot image activation
best_val_score = 'models-and-results-UTKface_cropped-balanced/...h5'
model_path     = 'models-and-results-UTKface_cropped-balanced/...h5'

plot_image_activation(model_path   = model_path, 
                      weights_path = best_val_score, 
                      img_path     = r'datasets\UTKface_cropped-new-structure-balanced\test\10\1_0_0_20170110212624891.jpg.chip.jpg')

<br><br><br><br><br><br><br><br>

# 2. ResNet-50 - Retrain ResNet-50 structure + own output layer

## 2.1 Import libaries, general functions and global variables

### 2.1.1 Libaries

In [1]:
# Import all the libaris in the module dependencies
from dependencies import *

Using TensorFlow backend.
C:\Users\Paulo\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Paulo\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Paulo\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users

### 2.1.2 General functions

In [2]:
# Import some general functions created by myself (loading the data, visualizing conv layers, etc.)
from general_functions import *

### 2.1.3 Global variables

In [3]:
dataset_path = os.path.join(os.getcwd(), 'datasets', 'UTKface_cropped-new-structure-balanced')

In [4]:
training_path   = os.path.join(os.getcwd(), 'datasets', 'UTKface_cropped-new-structure-balanced', 'training')
validation_path = os.path.join(os.getcwd(), 'datasets', 'UTKface_cropped-new-structure-balanced', 'validation')
test_path       = os.path.join(os.getcwd(), 'datasets', 'UTKface_cropped-new-structure-balanced', 'test')

In [5]:
df = load_dataset('df-UTKface_cropped-balanced.pkl')
df.head()

,age,age-binned,gender,race,datetime,category,path
0,10,"(0, 10]",0,0,20161220222308131,training,C:\Users\Paulo\Documents\GitHub\deep-learning-age-estimation\notebooks\datasets\UTKface_cropped-new-structure-balanced\training\10
1,10,"(0, 10]",0,0,20170103200329407,training,C:\Users\Paulo\Documents\GitHub\deep-learning-age-estimation\notebooks\datasets\UTKface_cropped-new-structure-balanced\training\10
2,10,"(0, 10]",0,0,20170103200522151,training,C:\Users\Paulo\Documents\GitHub\deep-learning-age-estimation\notebooks\datasets\UTKface_cropped-new-structure-balanced\training\10
3,10,"(0, 10]",0,0,20170103233459275,training,C:\Users\Paulo\Documents\GitHub\deep-learning-age-estimation\notebooks\datasets\UTKface_cropped-new-structure-balanced\training\10
4,10,"(0, 10]",0,0,20170104013211746,training,C:\Users\Paulo\Documents\GitHub\deep-learning-age-estimation\notebooks\datasets\UTKface_cropped-new-structure-balanced\training\10


In [6]:
df['age-binned'].value_counts()

(60, 70]    1170
(50, 60]    1170
(40, 50]    1170
(30, 40]    1170
(20, 30]    1170
(10, 20]    1170
(0, 10]     1170
Name: age-binned, dtype: int64

## 2.2 Generator

In [7]:
# Define image generators objects
train_datagen      = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen       = ImageDataGenerator(rescale=1./255)

# Create image generators
TARGET_SIZE = (224, 224) # We replicate the 224 x 224 shape like the paper.
COLOR_MODE  = 'rgb'      # Model ResNet-50 will expect rgb
BATCH_SIZE  = 32

train_generator    = train_datagen.flow_from_directory(directory   = training_path,
                                                       target_size = TARGET_SIZE,
                                                       color_mode  = COLOR_MODE,
                                                       batch_size  = BATCH_SIZE)

validation_datagen = train_datagen.flow_from_directory(directory   = validation_path,
                                                       target_size = TARGET_SIZE,
                                                       color_mode  = COLOR_MODE,
                                                       batch_size  = BATCH_SIZE)

test_generator     = test_datagen.flow_from_directory(directory    = test_path,
                                                      target_size  = TARGET_SIZE,
                                                      color_mode   = COLOR_MODE,
                                                      batch_size   = BATCH_SIZE)

Found 6177 images belonging to 7 classes.
Found 1629 images belonging to 7 classes.
Found 384 images belonging to 7 classes.


<br>

## 2.3 Import ResNet-50 stucture and add global averge pooling and output layer

In [44]:
TARGET_SIZE_WITH_DIMENSIONS = list(TARGET_SIZE)
TARGET_SIZE_WITH_DIMENSIONS.append(3)
TARGET_SIZE_WITH_DIMENSIONS = tuple(TARGET_SIZE_WITH_DIMENSIONS)

model = keras.applications.resnet.ResNet50(include_top = False, 
                                           weights     = None,  
                                           input_shape = TARGET_SIZE_WITH_DIMENSIONS)

In [45]:
# Adding GlobalAveragePooling2D and an output layer
x = model.output
x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x) # Could add dropout layer
predictions = Dense(7, activation= 'softmax')(x)
model = Model(inputs = model.input, outputs = predictions)

# Add compiler
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy']) 

In [46]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_9[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

<br>

## 2.4 Run the model

In [16]:
filepath = "models-and-results-UTKface_cropped-balanced/resnet-50-2.0-{val_accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit_generator(generator           = train_generator,
                              steps_per_epoch     = train_generator.n // train_generator.batch_size,
                              epochs              = 15,
                              validation_data     = validation_datagen,
                              validation_steps    = validation_datagen.n // validation_datagen.batch_size,
                              verbose             = 1,
                              callbacks           = callbacks_list,
                              workers             = (multiprocessing.cpu_count())
                             )

RuntimeError: You must compile your model before using it.

## 2.5 Save the results

In [ ]:
# Save results
model.save('models-and-results-UTKface_cropped-balanced/neural-network-resnet-50-2.0-model.h5')
save_history_model(history          = history,
                   filename         = 'neural-network-resnet-50-2.0-history', 
                   storage_location = 'models-and-results-UTKface_cropped-balanced')

In [ ]:
# Plot history
plot_history(history, line1='accuracy', line2='val_accuracy',
             x_as = 'Epochs', y_as = 'Accuracy', title = 'neural-network-2.0-resnet-50 Accuracy')

plot_history(history, line1='loss', line2='val_loss',
             x_as = 'Epochs', y_as = 'Loss', title = 'neural-network-2.0-resnet-50 Loss')

In [ ]:
# Plot image activation
best_val_score = 'models-and-results-UTKface_cropped-balanced/...h5'
model_path     = 'models-and-results-UTKface_cropped-balanced/...h5'

plot_image_activation(model_path   = model_path, 
                      weights_path = best_val_score, 
                      img_path     = r'datasets\UTKface_cropped-new-structure-balanced\test\10\1_0_0_20170110212624891.jpg.chip.jpg')